In [ ]:
import os 
import shutil
import pandas as pd

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# access data 
# %cd content/drive/MyDrive/data/raw/ # test
%cd /content/drive/MyDrive/Work/Frontline/Unzipped

/content/drive/MyDrive/Work/Frontline/Unzipped


In [31]:
# check number of files 
num_files = len(os.listdir("."))
print("Number of files in the folder: ", num_files)

Number of files in the folder:  257


## Extract Zip Files

In [17]:
#check number of zip files
assert len([name for name in os.listdir('.') if name.split(".")[-1]=="zip"])==253, "Number of zip files incorrect"

In [45]:
extracted=[name for name in os.listdir('.') if name.split(".")[-1]!="zip"]
zip_files=[name for name in os.listdir('.') if name.split(".")[-1]=="zip"]
for i in extracted:
  print(i)
  zip_files.remove(i+".zip")

OSZ


ValueError: ignored

In [ ]:
#unzip folders
for file in zip_files:
  if file.split(".")[-1]=="zip":   
    shutil.unpack_archive(file,file.split(".")[0])
    print("Unzipped {} sucessfully".format(file))
    zip_files.remove(file)

Unzipped SBIL.zip sucessfully
Unzipped STAG.zip sucessfully
Unzipped ZEIO.zip sucessfully
Unzipped LR.zip sucessfully
Unzipped DAZ.zip sucessfully
Unzipped NRU.zip sucessfully
Unzipped BDF.zip sucessfully
Unzipped PNP.zip sucessfully
Unzipped AWP.zip sucessfully
Unzipped CBIL.zip sucessfully
Unzipped HBTO.zip sucessfully
Unzipped WORZ.zip sucessfully
Unzipped ABGW.zip sucessfully
Unzipped RN.zip sucessfully
Unzipped ABMS.zip sucessfully
Unzipped KLEI.zip sucessfully
Unzipped NOW.zip sucessfully
Unzipped GAZ.zip sucessfully
Unzipped ABRM.zip sucessfully
Unzipped BKR.zip sucessfully
Unzipped RVZ.zip sucessfully
Unzipped PAZ.zip sucessfully
Unzipped LUXT.zip sucessfully
Unzipped ABKL.zip sucessfully
Unzipped BEOB.zip sucessfully
Unzipped OVZ.zip sucessfully
Unzipped SWT.zip sucessfully
Unzipped HK.zip sucessfully
Unzipped MUAZ.zip sucessfully
Unzipped NPHA.zip sucessfully
Unzipped HOZE.zip sucessfully
Unzipped KIRZ.zip sucessfully
Unzipped WOAZ.zip sucessfully
Unzipped MBVS.zip sucessfull

ReadError: ignored

In [ ]:
print(str(len(zip_files)) +" files were not unzipped")

165 files were not unzipped


In [ ]:
# check if any folder is empty
empty_folders = []

for root, dirs, files in os.walk('.'):
    if not dirs and not files:
        empty_folders.append(root)

print("Empty folders:")
for folder in empty_folders:
    print(folder)


Empty folders:


## Read Data

In [13]:
import pandas_read_xml as pdx
import pandas as pd

In [ ]:
dataset = pd.DataFrame(columns=['Filename', 'Type', 'Content'])

for entry in sorted(os.listdir('.')):
    entry_path = os.path.join('.', entry)
    if os.path.isdir(entry_path):
        # Entry is a folder, add XML files in the folder to the dataset
        for file in sorted(os.listdir(entry_path)):
            if file.endswith('.xml'):
                file_path = os.path.join(entry_path, file)
                with open(file_path, 'r') as f:
                    content = f.read()
                dataset = dataset.append({'Filename': file, 'Type': 'XML', 'Content': content}, ignore_index=True)
    elif os.path.isfile(entry_path) and entry.endswith('.xml'):
        # Entry is an XML file, add it to the dataset
        with open(entry_path, 'r') as f:
            content = f.read()
        dataset = dataset.append({'Filename': entry, 'Type': 'XML', 'Content': content}, ignore_index=True)

print(dataset.head())


In [37]:
def extract_content(df):
  # ressort, author, title not always an attribute in article
  df_all=pd.DataFrame(columns=["quelle","jahrgang","datum", "autor","ressort","titel","text" ])
  for art in df:
    quelle=art["metadaten"]["quelle"]["name"]
    jahrgang=art["metadaten"]["quelle"]["jahrgang"]
    datum=art["metadaten"]["quelle"]["datum"]
    try: 
      autor=art["metadaten"]["autor"]["autor-name"]
    except:
      autor=""
    try: 
      ressort=art["inhalt"]["titel-liste"]["ressort"]
    except:
      ressort=""
    try:
      titel=art["inhalt"]["titel-liste"]["titel"]
    except:
      titel=""
    text=art["inhalt"]["text"]
    df_all.loc[len(df_all),:]=[quelle, jahrgang,datum, autor, ressort, titel, text]
  return df_all


In [ ]:
review=[]
for extr_file in extracted[6:]:
  print(extr_file)
  df_all=pd.DataFrame(columns=["quelle","jahrgang","datum", "autor","ressort","titel","text" ])
  try:
    for xml in [i for i in os.listdir(extr_file) if i.split(".")[-1]=="xml"]:
      df= pdx.read_xml(extr_file+"/"+xml, ['artikel-liste',"artikel"]).iloc[0]
      df=extract_content(df)
      df_all=pd.concat([df_all,df])
    shutil.rmtree(extr_file)
    df_all.to_csv(extr_file+".csv",encoding="UTF8")
  except:
    review.append(extr_file)

things to do: 
- connect to github 
- create and store dataset 
- do some descriptive analysis: number of articles per newspaper, number of newspapers, number of topics per newspaper, etc. 
- do filtering: german newspapers only, DA related topics only 
- topic analysis: run the filtered dataset through a generic topic model 